# Sitzung 7

Diese Skripte sind ausschließlich als Zusatz-Material gedacht. Speziell für diejenigen unter Euch, die einen Einblick in das Programmieren gewinnen wollen. Wenn Du es also leid bist repetitive Tätigkeiten auszuführen und das lieber einer Maschine überlassen willst, bist Du hier genau richtig. 

<span style="color:red"> Die Codes sind nicht für die Klausur relevant, genau genommen haben sie mit dem Lehrstuhl für Statistik __rein gar nichts__ zu tun. </span>

---

In [46]:
import numpy as np
from seaborn import kdeplot
from matplotlib import pyplot as plt
from ipywidgets import interactive

## Der zentrale Grenzwertsatz

In [27]:
x = np.random.uniform(size=5)
x_bar = x.mean()
print("Werte von x:\n", x)
print("Mittelwert von x:", x_bar)

Werte von x:
 [0.01517029 0.53292296 0.98158077 0.33336742 0.19631933]
Mittelwert von x: 0.41187215500513047


In [28]:
X = np.random.uniform(size=(3, 5))
print(X)

[[0.1115667  0.98011276 0.30601222 0.95981864 0.91256209]
 [0.79240368 0.61864478 0.48331703 0.22517061 0.22329809]
 [0.62418115 0.89317627 0.36085467 0.520462   0.47902275]]


In [35]:
X.mean(axis=1)

array([0.65401448, 0.46856684, 0.57553937])

### Wie ist der Mittelwert $\bar{X}$ verteilt?

In [65]:
def mean_viewer(samples=3, sample_size=5):
    X = np.random.uniform(size=(samples, sample_size))
    # empirical
    X_bar = X.mean(axis=1)
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    kdeplot(X_bar.tolist(), label="empirische Dichte", ax=ax)
    
    # theoretical
    mu = X_bar.mean()
    sigma = np.std(X_bar)
    x = np.linspace(0, 1, 100)
    f_x = 1/np.sqrt(2*np.pi*sigma**2)*np.exp(-0.5*((x-mu)/sigma)**2)
    ax.plot(x, f_x, "--", c="red", alpha=0.5,
            label="$Z \sim N({:3.3}, {:3.3})$".format(mu, sigma))
    plt.title("Beispiel: zentraler Grenzwertsatz")
    plt.legend()
    
interactive(mean_viewer,
            samples=(2, 100000, 1),
            sample_size=(1, 1000, 1))

interactive(children=(IntSlider(value=3, description='samples', max=100000, min=2), IntSlider(value=5, descrip…